In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import torch as pt
import time
import sys

sys.path.insert(0, '..')

from NNsolver import NNSolver
from NNarchitectures import DenseNet_g
from problems import BondpriceMultidim
from utilities import get_X_process, plot_NN_evaluation, compute_PDE_loss

%load_ext autoreload
%autoreload 2

device = pt.device('cpu')

In [ ]:
T = 1
problem = BondpriceMultidim(d=100, T=T)
K = 1000
K_batch = 1000
print_every = 500
delta_t = 0.01
sq_delta_t = pt.sqrt(pt.tensor(delta_t))
N = int(np.ceil(T / delta_t))
gradient_steps = 15 * [2000] + (N - 16) * [300] + [10000]
learning_rates = [0.00005] * (N - 1) + [0.0001]

model = NNSolver(problem, 'BondpriceMultidim', learning_rates=learning_rates, gradient_steps=gradient_steps, NN_class=DenseNet_g, K=K, 
                 K_batch=K_batch, delta_t=delta_t, print_every=print_every, method='implicit')

model.Y_n = [DenseNet_g(problem.d, 1, lr=learning_rates[n], arch=[110, 110, 50, 50], problem=problem) for n in range(N)] + [problem.g]

In [ ]:
model.train()

In [ ]:
fig = plot_NN_evaluation(model, n=N-1, reference_solution=False, Y_0_true=0)

In [ ]:
pde_loss_eval = compute_PDE_loss(problem, delta_t=0.01, K=K, Y_n=model.Y_n, vfun=None, testOde=None, seed=44, print_every=10)

In [ ]:
np.mean(pde_loss_eval)